# 初期設定

## 1. Azure Machine Learning service Python SDK バージョン確認

In [13]:
import azureml.core

print("This notebook was created using version 1.0.23 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.0.23 of the Azure ML SDK
You are currently using version 1.0.23 of the Azure ML SDK


## 2. Azure Machine Learning service Workspaceの認証

接続情報を記録する config.json を作成
- subscription_id : サブスクリプションのID
- resource_group : リソースグループ名
- workspace_name : Azure Machine Learning service ワークスペース名称

In [14]:
from azureml.core import Workspace

try:
    ws = Workspace(
        subscription_id = "9c0f91b8-eb2f-484c-979c-15848c098a6b", 
        resource_group = "dllab", 
        workspace_name = "azureml"
    )

    ws.write_config()
    print("Workspace configuration succeeded.")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

Workspace configuration succeeded.


In [12]:
%%writefile config.json
{
    "subscription_id": "",
    "resource_group": "handson",
    "workspace_name": "azureml"
}

Overwriting config.json


In [9]:
ws.from_config()
print(ws.subscription_id, ws.resource_group, ws.name)

Found the config file in: /Users/konabuta/Project/Manufacturing-ML/.azureml/config.json
9c0f91b8-eb2f-484c-979c-15848c098a6b dllab azureml


## Python SDK アップデート

In [5]:
#!pip install --upgrade azureml-sdk[notebooks,automl,explain,contrib] azureml-dataprep

## クラスター構築

注意： 無償のサブスクリプションを使用されている人は、STANDARD_D2_V2 を使用してください

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# クラスター名の指定
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_NC6s_v3", ## GPU/CPUインスタンスの種類 
                                                       min_nodes=1, # 最小ノード数
                                                       max_nodes=1, # 最大ノード数
                                                       vm_priority='lowpriority') ## lowpriority = 低優先 | dedicated = 専用

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-03-26T16:53:01.785000+00:00', 'errors': None, 'creationTime': '2019-02-22T01:18:20.614322+00:00', 'modifiedTime': '2019-03-22T09:04:08.714135+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 10, 'nodeIdleTimeBeforeScaleDown': 'PT300S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_NC6'}
